<a href="https://colab.research.google.com/github/KonstantinElfimow/analytical-table-E-Commerce/blob/main/analytical_table_E_commerce_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Я буду использовать https://www.kaggle.com/carrie1/ecommerce-data . Я залил их на свой гугл диск, и тут скачиваю по ссылке

https://drive.google.com/file/d/1YpyajgsyZM07df7ibglOx5gjjqzwcE0P/view?usp=sharing

In [4]:
import pandas as pd
import numpy as np

In [5]:
!gdown --id 1YpyajgsyZM07df7ibglOx5gjjqzwcE0P

/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1YpyajgsyZM07df7ibglOx5gjjqzwcE0P
To: /content/archive.zip
100% 7.55M/7.55M [00:00<00:00, 77.3MB/s]


In [8]:
df = pd.read_csv('archive.zip', encoding='ISO-8859-1')

In [9]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


Тут транзакции по пользователям. Они покупают какие-то товары. Посмотрим, какой тут временной промежуток. И переведем дату из строки в формат даты.

In [11]:
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'], format='%m/%d/%Y %H:%M')

In [14]:
df['InvoiceDate'].min()

Timestamp('2010-12-01 08:26:00')

In [15]:
df['InvoiceDate'].max()

Timestamp('2011-12-09 12:50:00')

Примерно год. Теперь я хочу сделать отчет по продажам на каждый месяц.

# 1. Создаём таблицу с уникальными юзерами

В реальной жизни такая таблица обычно имеется сразу и можно ей пользоваться. Здесь у нас такой нет -- создадим сами. Но сначала посмотрим на пропущенные значения.

In [16]:
df.isna().mean()

InvoiceNo      0.000000
StockCode      0.000000
Description    0.002683
Quantity       0.000000
InvoiceDate    0.000000
UnitPrice      0.000000
CustomerID     0.249267
Country        0.000000
dtype: float64

У нас куча юзеров не заполнена. Если делать груп бай по NaN, то NaN просто не учитывается. Нам это не подходит. По этому заполним пропущенных юзеров значением -999999, чтобы этот юзер отображался в сводниках.

In [17]:
df['CustomerID'] = df['CustomerID'].fillna(-999999)

In [23]:
users = df.groupby(['CustomerID'])['InvoiceDate'].min().reset_index()

In [24]:
users.head()

,CustomerID,InvoiceDate
0,-999999.0,2010-12-01 11:52:00
1,12346.0,2011-01-18 10:01:00
2,12347.0,2010-12-07 14:57:00
3,12348.0,2010-12-16 19:09:00
4,12349.0,2011-11-21 09:51:00


In [25]:
users.columns = ['CustomerID', 'reg_date']

In [26]:
users['reg_month'] = users['reg_date'].values.astype('datetime64[M]')

In [29]:
users

,CustomerID,reg_date,reg_month
0,-999999.0,2010-12-01 11:52:00,2010-12-01
1,12346.0,2011-01-18 10:01:00,2011-01-01
2,12347.0,2010-12-07 14:57:00,2010-12-01
3,12348.0,2010-12-16 19:09:00,2010-12-01
4,12349.0,2011-11-21 09:51:00,2011-11-01
...,...,...,...
4368,18280.0,2011-03-07 09:52:00,2011-03-01
4369,18281.0,2011-06-12 10:53:00,2011-06-01
4370,18282.0,2011-08-05 13:35:00,2011-08-01
4371,18283.0,2011-01-06 14:14:00,2011-01-01


# 2. Сгенерируем таблицу, где на каждого юзера будет каждый месяц

In [30]:
min_month = df['InvoiceDate'].values.astype('datetime64[M]').min()

In [31]:
max_month = df['InvoiceDate'].values.astype('datetime64[M]').max()

In [32]:
(min_month, max_month)

(numpy.datetime64('2010-12'), numpy.datetime64('2011-12'))

In [33]:
pd.date_range(min_month,max_month,freq='MS')

DatetimeIndex(['2010-12-01', '2011-01-01', '2011-02-01', '2011-03-01',
               '2011-04-01', '2011-05-01', '2011-06-01', '2011-07-01',
               '2011-08-01', '2011-09-01', '2011-10-01', '2011-11-01',
               '2011-12-01'],
              dtype='datetime64[ns]', freq='MS')

In [34]:
dr = pd.DataFrame(pd.date_range(min_month,max_month,freq='MS'))

In [35]:
dr

,0
0,2010-12-01
1,2011-01-01
2,2011-02-01
3,2011-03-01
4,2011-04-01
5,2011-05-01
6,2011-06-01
7,2011-07-01
8,2011-08-01
9,2011-09-01


In [36]:
dr.columns = ['month']

Теперь надо сделать крос джоин. Должно получиться

In [38]:
len(users)*len(dr)

56849

строк. В пандас крос джоин делается так:

In [39]:
dr['key'] = 1
users['key'] = 1

In [40]:
report = dr.merge(users, on='key')

In [41]:
report.head()

,month,key,CustomerID,reg_date,reg_month
0,2010-12-01,1,-999999.0,2010-12-01 11:52:00,2010-12-01
1,2010-12-01,1,12346.0,2011-01-18 10:01:00,2011-01-01
2,2010-12-01,1,12347.0,2010-12-07 14:57:00,2010-12-01
3,2010-12-01,1,12348.0,2010-12-16 19:09:00,2010-12-01
4,2010-12-01,1,12349.0,2011-11-21 09:51:00,2011-11-01


In [42]:
len(report)

56849

Как и ожидали. Но у нас есть пользователи, которые пришли после минимальной даты (2010-12) и глупо хранить по ним строчки за тот период, пока их с нами не было.

In [43]:
report = report[report['month'] >= report['reg_month']]

In [44]:
len(report)

37449

Посмотрим на отдельного юзера

In [45]:
report[report['CustomerID'] == 12346.0]

,month,key,CustomerID,reg_date,reg_month
4374,2011-01-01,1,12346.0,2011-01-18 10:01:00,2011-01-01
8747,2011-02-01,1,12346.0,2011-01-18 10:01:00,2011-01-01
13120,2011-03-01,1,12346.0,2011-01-18 10:01:00,2011-01-01
17493,2011-04-01,1,12346.0,2011-01-18 10:01:00,2011-01-01
21866,2011-05-01,1,12346.0,2011-01-18 10:01:00,2011-01-01
26239,2011-06-01,1,12346.0,2011-01-18 10:01:00,2011-01-01
30612,2011-07-01,1,12346.0,2011-01-18 10:01:00,2011-01-01
34985,2011-08-01,1,12346.0,2011-01-18 10:01:00,2011-01-01
39358,2011-09-01,1,12346.0,2011-01-18 10:01:00,2011-01-01
43731,2011-10-01,1,12346.0,2011-01-18 10:01:00,2011-01-01


In [46]:
report[report['CustomerID'] == 12448.0]

,month,key,CustomerID,reg_date,reg_month
43814,2011-10-01,1,12448.0,2011-10-26 13:46:00,2011-10-01
48187,2011-11-01,1,12448.0,2011-10-26 13:46:00,2011-10-01
52560,2011-12-01,1,12448.0,2011-10-26 13:46:00,2011-10-01


Теперь можно джойнить инфу по продажам. Но сначала

# 3. Сгруппируем данные по продажам перед тем, как джойнить их к отчету.

Посчитаем суммарные продажи на каждый месяц.

In [47]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [48]:
df['month'] = df['InvoiceDate'].values.astype('datetime64[M]')

In [50]:
df['revenue'] = df['Quantity'] * df['UnitPrice']

Там есть отрицательные значения по количеству, наверное, это возвраты. Сейчас я не буду это учитывать

In [57]:
sales_month = df.groupby(['CustomerID', 'month']).agg({'revenue': ['sum']}).reset_index()
sales_month.columns = ['CustomerID','month','revenue']

In [60]:
sales_month.head()

,CustomerID,month,revenue
0,-999999.0,2010-12-01,194353.00
1,-999999.0,2011-01-01,84925.88
2,-999999.0,2011-02-01,61516.50
3,-999999.0,2011-03-01,103302.47
4,-999999.0,2011-04-01,67159.27


И здесь обязательно проверка по то тоталу, общей сумме по ревеню.

In [61]:
df['revenue'].sum()

9747747.933999998

In [62]:
sales_month['revenue'].sum()

9747747.934

Сходится. То что там не бьются далекие цифры полсе запятой это ок. https://stackoverflow.com/a/11873114/4527289

# 4. Джойним к отчету

In [63]:
report = report.merge(sales_month, how='left', on=['CustomerID', 'month'])

In [64]:
report.head()

,month,key,CustomerID,reg_date,reg_month,revenue
0,2010-12-01,1,-999999.0,2010-12-01 11:52:00,2010-12-01,194353.00
1,2010-12-01,1,12347.0,2010-12-07 14:57:00,2010-12-01,711.79
2,2010-12-01,1,12348.0,2010-12-16 19:09:00,2010-12-01,892.80
3,2010-12-01,1,12370.0,2010-12-14 12:58:00,2010-12-01,1868.02
4,2010-12-01,1,12377.0,2010-12-20 09:37:00,2010-12-01,1001.52


проверка по котрольной сумме.

In [65]:
report['revenue'].sum()

9747747.934

In [67]:
df['revenue'].sum()

9747747.933999998

Посмотрим на отдельного клиента.

In [68]:
report[report['CustomerID'] == 12347.0]

,month,key,CustomerID,reg_date,reg_month,revenue
1,2010-12-01,1,12347.0,2010-12-07 14:57:00,2010-12-01,711.79
951,2011-01-01,1,12347.0,2010-12-07 14:57:00,2010-12-01,475.39
2321,2011-02-01,1,12347.0,2010-12-07 14:57:00,2010-12-01,NaN
4071,2011-03-01,1,12347.0,2010-12-07 14:57:00,2010-12-01,NaN
6261,2011-04-01,1,12347.0,2010-12-07 14:57:00,2010-12-01,636.25
8750,2011-05-01,1,12347.0,2010-12-07 14:57:00,2010-12-01,NaN
11518,2011-06-01,1,12347.0,2010-12-07 14:57:00,2010-12-01,382.52
14521,2011-07-01,1,12347.0,2010-12-07 14:57:00,2010-12-01,NaN
17715,2011-08-01,1,12347.0,2010-12-07 14:57:00,2010-12-01,584.91
21076,2011-09-01,1,12347.0,2010-12-07 14:57:00,2010-12-01,NaN


Вроде ок. Видно, что активность этого клиента нерегулярная! Например у него есть перерыв в покупках после 2011-01-01 на два месяца.

# 5. Базовые метрики

С помощью этой таблицы можно расчитать кучу всего полезного.

Продажи на каждый месяц

In [69]:
report.groupby('month')[['revenue']].agg('sum').reset_index()

,month,revenue
0,2010-12-01,748957.020
1,2011-01-01,560000.260
2,2011-02-01,498062.650
3,2011-03-01,683267.080
4,2011-04-01,493207.121
5,2011-05-01,723333.510
6,2011-06-01,691123.120
7,2011-07-01,681300.111
8,2011-08-01,682680.510
9,2011-09-01,1019687.622


Добавим флаг юзера, нового юзера, и активного юзера.

In [70]:
report['user'] = 1
report['new'] = (report['reg_month'] == report['month']) * 1
report['active'] = (report['revenue'] > 0) * 1

In [71]:
report.head()

,month,key,CustomerID,reg_date,reg_month,revenue,user,new,active
0,2010-12-01,1,-999999.0,2010-12-01 11:52:00,2010-12-01,194353.00,1,1,1
1,2010-12-01,1,12347.0,2010-12-07 14:57:00,2010-12-01,711.79,1,1,1
2,2010-12-01,1,12348.0,2010-12-16 19:09:00,2010-12-01,892.80,1,1,1
3,2010-12-01,1,12370.0,2010-12-14 12:58:00,2010-12-01,1868.02,1,1,1
4,2010-12-01,1,12377.0,2010-12-20 09:37:00,2010-12-01,1001.52,1,1,1


In [72]:
report.groupby('month')[['revenue','user','new','active']].agg('sum').reset_index()

,month,revenue,user,new,active
0,2010-12-01,748957.020,949,949,882
1,2011-01-01,560000.260,1370,421,737
2,2011-02-01,498062.650,1750,380,754
3,2011-03-01,683267.080,2190,440,969
4,2011-04-01,493207.121,2489,299,851
5,2011-05-01,723333.510,2768,279,1053
6,2011-06-01,691123.120,3003,235,987
7,2011-07-01,681300.111,3194,191,944
8,2011-08-01,682680.510,3361,167,931
9,2011-09-01,1019687.622,3659,298,1261


Средние продажи на клиента

In [73]:
t = report.groupby('month')[['revenue','user','new','active']].agg('sum')

In [74]:
t['sales_per_customer'] = t['revenue'] / t['active']

In [75]:
t

,revenue,user,new,active,sales_per_customer
month,,,,,
2010-12-01,748957.020,949,949,882,849.157619
2011-01-01,560000.260,1370,421,737,759.837531
2011-02-01,498062.650,1750,380,754,660.560544
2011-03-01,683267.080,2190,440,969,705.125986
2011-04-01,493207.121,2489,299,851,579.561834
2011-05-01,723333.510,2768,279,1053,686.926410
2011-06-01,691123.120,3003,235,987,700.226059
2011-07-01,681300.111,3194,191,944,721.716219
2011-08-01,682680.510,3361,167,931,733.276595
